In [1]:
import numpy as np
import copy
import pickle
import wntr
import wntr.network.controls as controls
from itertools import combinations

In [2]:
def N_K_simulation(nk_pipe_list, wdn, conting_st, conting_et, seriest):
    # Get the nodes with demands
    junctions = wdn.junction_name_list
    nodes = []
    for j in junctions:
        j_object = wdn.get_node(j)
        base_demand = j_object.demand_timeseries_list[0].base_value
        if base_demand > 1e-8:
            nodes.append(j)
    
    # Perform simulation for normal operations
    wdno = copy.deepcopy(wdn)
    wdno.options.time.duration = 24 * 3600
    wdno.options.hydraulic.demand_model = 'PDD'
    simo = wntr.sim.EpanetSimulator(wdno)
    resultso = simo.run_sim() # by default run EPANET 2.2
    demandso = resultso.node['demand']
    # Calculate the expected total demand
    expected_total = {}
    for t in seriest:
        expected_total[t] = 0
        for n in nodes:
            expected_total[t] += demandso.loc[t*3600, n]
    
    # N-k contingency simulation
    functionality_loss = {}
    # The corresponding simulation results after turning off a set of pipe
    for nk_pipe in nk_pipe_list:
        # Initialization
        functionality_loss[nk_pipe] = {}
        for t in seriest:
            functionality_loss[nk_pipe][t] = 1
        
        # Make a copy of the original water distribution network
        wdnc = copy.deepcopy(wdn)
        ctrl1_list = []
        ctrl2_list = []
        for k in range(len(nk_pipe)):
            p_object = wdnc.get_link(nk_pipe[k])
            p_act1 = controls.ControlAction(p_object, 'status', 0)
            p_cond1 = controls.SimTimeCondition(wdnc, '=', str(conting_st) + ':00:00')
            ctrl1 = controls.Control(p_cond1, p_act1)
            ctrl1_list.append(ctrl1)
            p_act2 = controls.ControlAction(p_object, 'status', 1)
            p_cond2 = controls.SimTimeCondition(wdnc, '=', str(conting_et) + ':00:00')
            ctrl2 = controls.Control(p_cond2, p_act2)
            ctrl2_list.append(ctrl2)
        # Assign the controls on the network
        for m in range(len(ctrl1_list)):
            wdnc.add_control('Conting_start'+str(m), ctrl1_list[m])
            wdnc.add_control('Conting_end'+str(m), ctrl2_list[m])
        
        wdnc.options.time.duration = conting_et * 3600
        wdnc.options.hydraulic.demand_model = 'PDD'
        simc = wntr.sim.WNTRSimulator(wdnc)
        try:
            resultsc = simc.run_sim() # runs EPANET 2.2.0  version = 2.2
            actual_demands = resultsc.node['demand']
            for t in seriest:
                # some simulations may only have part of results 
                try:
                    actual_total = 0
                    for n in nodes:
                        actual_total += actual_demands.loc[t*3600, n]
                    loss = 1 - (actual_total / expected_total[t])
                    functionality_loss[nk_pipe][t] = loss
                except:
                    pass
        except:
            pass
    return functionality_loss

In [3]:
Wdn = wntr.network.WaterNetworkModel('Net3S.inp')
# The time of the contingency 
Conting_st = 0
# The end of the simulation
Simula_et = 24
Seriest = [int(t) for t in np.linspace(1, 23, 23)]
Pipe_list = Wdn.pipe_name_list

In [4]:
# Use the pipes that are in the two core
K = 3
Nk_pipe_list = list(combinations(Pipe_list, K))

In [5]:
len(Nk_pipe_list)

227920

In [6]:
Pipe_off_list = Nk_pipe_list[200000: ]
# Pipe_off_list = Nk_pipe_list 
Off_results = N_K_simulation(Pipe_off_list, Wdn, Conting_st, Simula_et, Seriest)

C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 03:43:12. Reached maximum number of iterations: 2999
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 03:39:35. Reached maximum number of iterations: 2999
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 03:38:50. Reached maximum number of iterations: 2999
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 05:00:00. Reached maximum number of iterations: 2999
  warnings.warn('Simulation did not converge at time ' + self._get_ti

C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 02:15:00. Line search failed at iteration 19
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 01:00:00. Line search failed at iteration 28
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 02:00:00. Line search failed at iteration 14
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 01:00:00. Line search failed at iteration 26
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz6

C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 02:45:00. Reached maximum number of iterations: 2999
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 01:15:00. Reached maximum number of iterations: 2999
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 00:00:00. Line search failed at iteration 60
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 01:00:00. Reached maximum number of iterations: 2999
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '

C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 01:00:00. Line search failed at iteration 33
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 00:00:00. Line search failed at iteration 39
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 01:00:00. Line search failed at iteration 19
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 01:30:31. Reached maximum number of iterations: 2999
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\U

C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 00:00:00. Line search failed at iteration 66
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 02:22:10. Line search failed at iteration 59
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 02:03:40. Line search failed at iteration 54
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 14:00:00. Line search failed at iteration 21
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz6

...

C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 02:22:11. Line search failed at iteration 53
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 02:03:56. Line search failed at iteration 54
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 02:22:14. Line search failed at iteration 48
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 02:03:55. Line search failed at iteration 49
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz6

C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 15:00:00. Line search failed at iteration 75
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 15:00:00. Line search failed at iteration 65
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 15:00:00. Line search failed at iteration 66
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 15:00:00. Line search failed at iteration 76
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz6

C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 15:00:00. Line search failed at iteration 60
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 15:00:00. Line search failed at iteration 59
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 15:00:00. Line search failed at iteration 64
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 15:00:00. Line search failed at iteration 62
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz6

C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 03:31:58. Reached maximum number of iterations: 2999
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 02:35:15. Reached maximum number of iterations: 2999
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 16:00:00. Line search failed at iteration 42
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 20:00:00. Line search failed at iteration 39
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + me

C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 17:00:00. Line search failed at iteration 21
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 19:00:00. Reached maximum number of iterations: 2999
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 03:38:24. Reached maximum number of iterations: 2999
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\xz68\Anaconda3\lib\site-packages\wntr\sim\core.py:904: UserWarning: Simulation did not converge at time 16:00:00. Line search failed at iteration 31
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + me

In [7]:
FOff_results = open('Net3S_N' + str(K) + '_part3.pickle','wb')
pickle.dump(Off_results, FOff_results)
FOff_results.close()